# Predicting Enron Spam Emails using Supervised Learning

## DS-GA 1001: Introduction to Data Science Final Project

### Scripts

## Supervised Learning Models

Created On: 11/30/2020

Modified On: 12/02/2020

### Description

This script establishes various supervised learning models for the `emails_cleaned.csv` dataset.

### Data

We applied feature engineering to make the data ready for models.

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import decomposition
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, mean_squared_error

print("SUCCESS! All modules have been imported.")

SUCCESS! All modules have been imported.


In [2]:
df = pd.read_csv('../data/emails_cleaned.csv')

In [3]:
# Remove rows containing missing values
df.dropna(subset=['X'], inplace=True)
# Confirm that there is no missing values
df.isnull().sum()

X    0
y    0
dtype: int64

In [4]:
df.shape
print('The model-ready dataset contains {} rows.'.format(df.shape[0]))

The model-ready dataset contains 785579 rows.


### Feature Engineering

We applied [Term frequency–inverse document frequency](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) (TF-IDF) method to transform string email contents into meaningful numeric figures so that they can be applied for model training.

#### TF-IDF

In [5]:
# Create a vectorization matrix using tf-idf vectorizer
vectorizer = TfidfVectorizer()
vectorized_emails = vectorizer.fit_transform(df.X)
vectorized_emails

<785579x143084 sparse matrix of type '<class 'numpy.float64'>'
	with 4292928 stored elements in Compressed Sparse Row format>

### Methods

#### Train Test Split

Before fitting the model, we splitted the dataset into two parts: train set and test set. We used train set to train models and used test set to examine the performance of each model.

In [6]:
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(vectorized_emails, df.y, test_size=test_size, random_state=88)

#### Grid Search Cross Validation

We also applied grid search with cross validation to tune the hyperparameter of the logistic regression, `C`. The goal is to find the optimal hyperparameter `C` so that our model will reach the optimal complexity.

### Logstic Regression with Elastic Net (Baseline)

We used logistic regression as our baseline model. We also applied elastic net to weight coefficients and added a penalty term to our model. 

##### Elastic Net

We first applied grid search on the parameter of the elastic net, `l1 ratio`. We created a hyperparameter space that contains 30 possible l1 ratio values. We then fitted the training data with 5-fold cross validation to get the best l1 ratio. 

In [16]:
# Setup a hyperparameter grid for l1_ratio that is from 0 to 1
l1_space = [0.2, 0.5, 0.8]
param_grid = {'l1_ratio': l1_space}

In [18]:
elastic_net = ElasticNet()

In [19]:
# Setup the grid search and fit the training data
gm_cv = GridSearchCV(elastic_net, param_grid, cv=5, n_jobs=-1)
gm_cv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=ElasticNet(), n_jobs=-1,
             param_grid={'l1_ratio': [0.2, 0.5, 0.8]})

In [20]:
# Predict on the test dataset and compute metrics
y_pred = gm_cv.predict(X_test)
r2 = gm_cv.score(X_test, y_test)
mse = mean_squared_error(y_test, y_pred)
print("Tuned ElasticNet l1 ratio: {}".format(gm_cv.best_params_))
print("Tuned ElasticNet R squared: {}".format(r2))
print("Tuned ElasticNet MSE: {}".format(mse))

Tuned ElasticNet l1 ratio: {'l1_ratio': 0.2}
Tuned ElasticNet R squared: -9.747972842921726e-06
Tuned ElasticNet MSE: 0.2497127785371497


In [27]:
best_l1_ratio = gm_cv.best_params_['l1_ratio']

#### Regularization Strength

We then applied another grid search method on the regularization parameter, `C`, of the logstic regression. A large `C` leads to overfit while a small `C` can get to underfit.

In [7]:
# Setup a hyperparameter grid for C that is from 0 to 1
c_space = [0.01, 0.1, 1, 10, 100]
param_grid = {'C': c_space}

In [8]:
# Fit a logistic regression model with elastic net
logreg = LogisticRegression(solver='saga', penalty='elasticnet', l1_ratio=0.2, max_iter=5000, n_jobs=-1)

In [ ]:
logreg_cv = GridSearchCV(logreg, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
logreg_cv.fit(X_train, y_train)

In [ ]:
best_param_logreg = logreg_cv.best_params_
validation_auc_logreg = logreg_cv.best_score_

y_pred = logreg_cv.predict_proba(x_test)[:, 1]
test_auc_logreg = roc_auc_score(y_test, y_pred)

In [ ]:
logreg_cv.best_estimator_

In [ ]:
print('Tuned Logistic Regression Parameters: {}'.format(best_param_logreg)
print('Tuned Logstic Regression Validation AUC: {}'.format(validation_auc_logreg)
print('Tuned Logstic Regression Test AUC: {}'.format(test_auc_logreg)

### Random Forest

In [ ]:
# Setup a hyperparamter grid for random forest
n_estimators = [50, 100, 200, 500]
max_depth = [10, 50, 100, 200]
min_samples_leaf = [10, 20 50, 100]
param_grid = {'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_leaf': min_samples_leaf}

In [ ]:
rf = RandomForestClassifier(criterion='entropy', n_jobs=-1)

In [ ]:
rf_cv = GridSearchCV(rf, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
rf_cv.fit(X_train, y_train)

In [ ]:
best_param_rf = rf_cv.best_params_
validation_auc_rf = logreg_cv.best_score_

y_pred = rf_cv.predict_proba(x_test)[:, 1]
test_auc_rf = roc_auc_score(y_test, y_pred)

In [ ]:
rf_cv.best_estimator_

In [ ]:
print('Tuned Random Forest Parameters: {}'.format(best_param_rf)
print('Tuned Random Forest Validation AUC: {}'.format(validation_auc_rf)
print('Tuned Random Forest Test AUC: {}'.format(test_auc_rf)

### Gradient Boosting

In [ ]:
# Setup a hyperparameter grid for gradient boosting
n_estimators = [100, 500]
max_depth = [10, 100]
param_grid = {'n_estimators': n_estimators, 'max_depth': max_depth}

In [ ]:
gboost = GradientBoostingClassifier()

In [ ]:
gboost_cv = GridSearchCV(gboost, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
gboost_cv.fit(X_train, y_train)

In [ ]:
best_param_gboost = gboost_cv.best_params_
validation_auc_gboost = gboost_cv.best_score_

y_pred = gboost_cv.predict_proba(x_test)[:, 1]
test_auc_gboost = roc_auc_score(y_test, y_pred)

In [ ]:
gboost_cv.best_estimator_

In [ ]:
print('Tuned Gradient Boosting Parameters: {}'.format(best_param_gboost)
print('Tuned Gradient Boosting Validation AUC: {}'.format(validation_auc_gboost)
print('Tuned Gradient Boosting Test AUC: {}'.format(test_auc_gboost)

### Multilayer Perceptron

In [ ]:
# Setup a hyperparameter grid for multilayer perceptron
alpha = [0.01, 0.1, 1, 10]
param_grid = {'alpha': alpha}

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(100, 30), batch_size=32)

In [ ]:
mlp_cv = GridSearchCV(mlp, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
mlp_cv.fit(X_train, y_train)

In [ ]:
best_param_mlp = mlp_cv.best_params_
validation_auc_mlp = mlp_cv.best_score_

y_pred = mlp_cv.predict_proba(x_test)[:, 1]
test_auc_mlp = roc_auc_score(y_test, y_pred)

In [ ]:
mlp_cv.best_estimator_

In [ ]:
print('Tuned Multilayer Perceptron Parameters: {}'.format(best_param_mlp)
print('Tuned Multilayer Perceptron Validation AUC: {}'.format(validation_auc_mlp)
print('Tuned Multilayer Perceptron Test AUC: {}'.format(test_auc_mlp)